<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#¿Qué-son-los-problemas-de-clasificación-de-Clases-desequilibradas?-(imbalanced-data)" data-toc-modified-id="¿Qué-son-los-problemas-de-clasificación-de-Clases-desequilibradas?-(imbalanced-data)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>¿Qué son los problemas de clasificación de Clases desequilibradas? (imbalanced data)</a></span></li><li><span><a href="#¿Cómo-nos-afectan-los-datos-desbalanceados?" data-toc-modified-id="¿Cómo-nos-afectan-los-datos-desbalanceados?-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>¿Cómo nos afectan los datos desbalanceados?</a></span></li><li><span><a href="#Manejo-del-desbalanceo-de-las-categorías" data-toc-modified-id="Manejo-del-desbalanceo-de-las-categorías-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Manejo del desbalanceo de las categorías</a></span><ul class="toc-item"><li><span><a href="#Downsampling" data-toc-modified-id="Downsampling-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Downsampling</a></span></li><li><span><a href="#Upsampling" data-toc-modified-id="Upsampling-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Upsampling</a></span></li><li><span><a href="#Upsampling-y-downsampling-con-la-librería--imblearn" data-toc-modified-id="Upsampling-y-downsampling-con-la-librería--imblearn-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Upsampling y downsampling con la librería  <code>imblearn</code></a></span><ul class="toc-item"><li><span><a href="#Downsampling-con-imblearn" data-toc-modified-id="Downsampling-con-imblearn-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Downsampling con imblearn</a></span></li><li><span><a href="#Upsampling-con-imblearn" data-toc-modified-id="Upsampling-con-imblearn-3.3.2"><span class="toc-item-num">3.3.2&nbsp;&nbsp;</span>Upsampling con imblearn</a></span></li></ul></li></ul></li></ul></div>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = None


import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler


from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

ModuleNotFoundError: No module named 'pandas'

In [2]:
!pip3 install pandas

Para la clase de hoy tendremos que instalarnos 

```
pip install imblearn

o 

conda install -c conda-forge imbalanced-learn

```

# ¿Qué son los problemas de clasificación de Clases desequilibradas? (imbalanced data)


En los problemas de clasificación en donde tenemos que etiquetar por ejemplo entre “spam” o “not spam” ó entre múltiples categorías (coche, barco, avión) solemos encontrar que en nuestro conjunto de datos contamos con que alguna de las clases de muestra es una clase “minoritaria” es decir, de la cual tenemos muy poquitas muestras. Esto provoca un desbalanceo en los datos. 

Un caso evidente es en el área de Salud en donde solemos encontrar conjuntos de datos con miles de registros con pacientes “negativos” y unos pocos casos positivos es decir, que padecen la enfermedad que queremos clasificar.

# ¿Cómo nos afectan los datos desbalanceados?

Por lo general afecta a los algoritmos en su proceso de generalización de la información y perjudicando a las clases minoritarias. Esto suena bastante razonable: si a una red neuronal le damos 990 de fotos de gatitos y sólo 10 de perros, no podemos pretender que logre diferenciar una clase de otra. Lo más probable que la red se limite a responder siempre “tu foto es un gato” puesto que así tuvo un acierto del 99% en su fase de entrenamiento.

Desde Python, podremos soluciar estos problemas usando distintas aproximaciones, forma parte esencial de la parte de preprocesamiento de datos antes de realizar nuestras predicciones. Algunas de las más importantes son: 

- Downsampling


- Upsampling

- 

Para este jupyter vamos a usar un nuevo set de datos. En este caso, cada fila representa un cliente, cada columna contiene los atributos del cliente. El conjunto de datos incluye información sobre

- Clientes que se han dado de baja en el último mes - la columna se llama Churn. Tiene valores de Si/No


- Servicios que cada cliente ha contratado: teléfono, líneas múltiples, Internet, seguridad en línea, copia de seguridad en línea, protección de dispositivos, soporte técnico y transmisión de TV y películas


- Información de la cuenta del cliente: cuánto tiempo ha sido cliente, contrato, método de pago, facturación sin papel, cargos mensuales y cargos totales


- Información demográfica sobre los clientes: sexo, rango de edad y si tienen pareja y personas a su cargo



In [ ]:
df = pd.read_csv("data/customer_churn.csv")
df.head()

Exploremos un poco nuestra variable respuesta, la que queremos predecir:

In [ ]:
df['Churn'].value_counts()

Como podemos ver, en nuestro dataset tenemos muchos No y pocos Si. Veamoslo graficamente:

In [ ]:
sns.set_theme(style="darkgrid")
sns.countplot(data = data, x= "Churn");

# Manejo del desbalanceo de las categorías

## Downsampling

Lo ideal sería tener el mismo número de muestras de ambas clases, pero la realidad es que esto nos va a pasar pocas veces en nuestra vida como analistas. En este tipo de aproximación, desechamos una serie de muestras de forma aleatoria de la clase mayoritaria hasta que tenemos la misma cantidad de muestras de cada clase. 

Este enfoque puede ser peligroso por varias razones:

- Puede que dentro de la clase dominante haya diferentes "grupos" de muestras de los que no somos conscientes. Por lo tanto, si simplemente descartamos muestras al azar corremos el riesgo de subrepresentar uno de los grupos o de alterar las proporciones de muestras de cada grupo.



- Si la cantidad de muestras de la clase minoritaria es extremadamente pequeña, acabaremos con un conjunto de datos muy reducido que será demasiado pequeño para entrenar un modelo y, por tanto, no podremos confiar en él.



In [ ]:
num_minoritarios = data["Churn"].value_counts()["Yes"]
num_minoritarios

In [ ]:
# nos creamos un dataframe solo con las filas donde la variable respuesta sea Yes. 
minoritarios = data[data["Churn"] == "Yes"]
minoritarios.head(2)


In [ ]:
# extraemos una muestra de la categoría mayoritaria que sea del mismo tamaño que la clase minotaria

mayoritarios = data[data["Churn"] == "No"].sample(num_minoritarios, random_state = 42)
mayoritarios.head(2)

In [ ]:
# Ahora es el momento de unir los *dataframes*
balanceado = pd.concat([minoritarios,mayoritarios],axis = 0)
balanceado.head(2)

In [ ]:
# chequeamos como es nuestro dataframe ahora
balanceado["Churn"].value_counts()

## Upsampling

Vamos a hacer un upsample de la clase minoritaria repitiendo las muestras de la clase minoritaria. Al contrario que en el caso anterior, si realizamos un muestreo ascendente estamos repitiendo muestras de la clase minoritaria hasta que terminemos con la misma cantidad de ambas clases.

Problemas que nos podemos encontrar cuando usamos este método: 

- Al repetir las muestras, no estamos añadiendo nueva información. Y el modelo resultante probablemente no será capaz de clasificar correctamente una muestra no vista de la clase minoritaria que no tenga una alta similitud con las muestras de nuestro conjunto de datos.

Para ello, podemos seguir utilizando la función `sample()` de Panda. Sin embargo, como vamos a repetir las muestras, necesitamos establecer la opción `replace=True` ( que por defecto está establecida en `False`).


In [ ]:
num_mayoritarios = data["Churn"].value_counts()["No"]
num_mayoritarios

In [ ]:
# seleccionamos ls datos de la clase mayoritaria

mayoritarios2 = data[data["Churn"]== "No"]
mayoritarios2.head(2)

In [ ]:
# hacemos un selección aleatoria de datos de la clase minoritaria, para tener el mismo número que la clase mayoritaria

minoritarios2 =data[data["Churn"]=="Yes"].sample(num_mayoritarios, replace=True)
minoritarios2.head(2)

In [ ]:
# unimos los dos dataframes

balanceado2 = pd.concat([mayoritarios2,minoritarios2], axis = 0)
balanceado2.head(2)

In [ ]:
balanceado2["Churn"].value_counts()

## Upsampling y downsampling con la librería  `imblearn`


En lugar de hacerlo manualmente, podemos utilizar la biblioteca `imblearn`. [Aquí](https://imbalanced-learn.org/stable/) os dejamos la documentación oficial por si quereis trastear con ella un poco más. 

### Downsampling con imblearn

In [ ]:
# separamos los datos en X e y. Siendo X las variables predictoras e y  la variable a predecir. 

X = data[["tenure","SeniorCitizen","MonthlyCharges"]]
y = data["Churn"]

In [ ]:
# iniciamos el método

down = RandomUnderSampler()

In [ ]:
# ajustamos el modelo
X_down, y_down = down.fit_resample(X,y)

In [ ]:
X_down.head()

In [ ]:
y_down.head()

Si chequeamos los valores ahora...

In [ ]:
y_down.value_counts()

### Upsampling con imblearn

In [ ]:
# iniciamos el método

upp = RandomOverSampler()

In [ ]:
# como siempre, ajustamos el modelo

X_upp, y_upp = upp.fit_resample(X, y)

In [ ]:
y_upp.value_counts()